In [14]:
import pandas as pd
import numpy as np
import datetime

In [37]:
# data_path = "../data/data_investeringer.xlsx"
data_path = "../data/full_data.xlsx"
df = pd.read_excel(data_path)
df.head()

,Kommune,ISIN kode,Værdipapirets navn,Udsteder,Markedsværdi (DKK),Type,ISIN,Selskab,Årsag til eksklusion,Selskab_normalized,Matched Udsteder,Matched Værdipapirets navn,Kommuner,Eksklusionsårsager,OBS_Type,Priority,Problematisk ifølge:,Sortlistet,Problemkategori
0,Rødovre,AEA000801018,Abu Dhabi Islamic Bank PJSC,Abu Dhabi Islamic Bank PJSC,4420.591824,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
1,Holstebro,AEA002001013,Aldar Properties,Aldar Properties,48912.517111,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,Fanø,AEA002001013,Aldar Properties PJSC,United Arab Emirates,56.605770,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,Fanø,AEE000301011,Emaar Properties PJSC,United Arab Emirates,396.240390,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,Fanø,AEE000401019,Emirates Telecommunications Gr,United Arab Emirates,396.240390,Aktie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [38]:
df_odense = df[df['Kommune'] == 'Odense']
df_odense = df_odense[['ISIN kode', 'Værdipapirets navn', 'Markedsværdi (DKK)']]

In [65]:
file_kommunale_investeringer = '../data/Kommunale investeringer - five big ones.xlsx'

# Load the specific sheet 'ODENSE OK' from 'Kommunale investeringer - five big ones'
sheet_name = 'ODENSE OK'

# Read the Excel files
df_odense_ok = pd.read_excel(file_kommunale_investeringer, sheet_name=sheet_name)
df_odense_ok.drop(['Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5'], axis=1, inplace=True)
# Remove duplicates based on the combination of 'ISIN', 'Beholdning', and 'Markedsværdi (DDK)'
# df_odense_ok_cleaned = df_odense_ok.drop_duplicates(subset=['ISIN', 'Beholdning', 'Markedsværdi (DDK)'], keep='first')

# Display the first few rows of the cleaned dataframe
# print(df_odense_ok_cleaned.head())

# Find duplicates based on 'Markedsværdi (DDK)' and 'Markedsværdi (DKK)' columns
duplicates = df_odense_ok[df_odense_ok.duplicated(subset=['ISIN', 'Beholdning', 'Markedsværdi (DDK)'], keep=False)]

# Display the duplicate rows
print(duplicates)



Empty DataFrame
Columns: [ISIN, Beholdning, Markedsværdi (DDK)]
Index: []


In [61]:
import numpy as np

df_odense_ok['Markedsværdi (DDK) original'] = df_odense_ok['Markedsværdi (DDK)']
# First, convert 'Markedsværdi (DDK)' and 'Markedsværdi (DKK)' to float and handle errors by setting invalid entries to NaN
def to_float_safe(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return np.nan

# Apply the conversion to both dataframes before merging
df_odense_ok['Markedsværdi (DDK)'] = df_odense_ok['Markedsværdi (DDK)'].apply(to_float_safe)
df_odense['Markedsværdi (DKK)'] = df_odense['Markedsværdi (DKK)'].apply(to_float_safe)

# Perform the merge
df_merged = pd.merge(
    df_odense_ok, 
    df_odense, 
    left_on=['ISIN', 'Beholdning', 'Markedsværdi (DDK)'], 
    right_on=['ISIN kode', 'Værdipapirets navn', 'Markedsværdi (DKK)'], 
    how='outer',  # Use outer merge to keep all rows even if they don't match
    suffixes=('_ok', '_odense')  # Add suffixes to differentiate between columns from both dataframes
)

# Create a new column 'Mismatch' to indicate if 'Markedsværdi (DDK)' and 'Markedsværdi (DKK)' do not match
df_merged['Mismatch'] = np.where(df_merged['Markedsværdi (DDK)'] != df_merged['Markedsværdi (DKK)'], 'Mismatch', 'Match')

# Filter for rows where there is a mismatch
mismatched_rows = df_merged[df_merged['Mismatch'] == 'Mismatch']

# Display the mismatched rows
print(mismatched_rows[['ISIN', 'Beholdning', 'Markedsværdi (DDK)', 'Markedsværdi (DKK)', 'Mismatch']])


              ISIN                                 Beholdning  \
50    BE6349605822          Warehouses De Pauw CVA Right 2024   
75    CA21037X1345             Constellation Software Inc/Can   
250   CH1326854028                          SGS SA Right 2024   
523   DK0004624624          1.00% Realkredit Danmark A/S 2025   
627   DK0007901763          5.00% Nykredit Realkredit AS 2029   
809   DK0009535478          3.00% Nykredit Realkredit AS 2053   
1152           NaN                                        NaN   
1153  FR001400P9F6               Covivio SA/France Right 2024   
1225  GB00B7FC0762                                 NMC Health   
1458           N/S                                 Main (DKK)   
1459           N/S                                 Main (DKK)   
1460           N/S                                 Main (DKK)   
1461           N/S                               Normal (GBP)   
1462           N/S                               Normal (USD)   
1617  US003CVR0169       

In [63]:
df_merged.to_excel('forskel_i_odense_data_3.xlsx')

In [45]:
# # Strip white spaces from the relevant columns in both dataframes
# df_odense_ok['ISIN'] = df_odense_ok['ISIN'].str.strip()
# df_odense_ok['Beholdning'] = df_odense_ok['Beholdning'].str.strip()

# df_odense['ISIN kode'] = df_odense['ISIN kode'].str.strip()
# df_odense['Værdipapirets navn'] = df_odense['Værdipapirets navn'].str.strip()

# # Perform the merge on 'ISIN kode' with 'ISIN', and 'Værdipapirets navn' with 'Beholdning'
# df_merged = pd.merge(
#     df_odense_ok, 
#     df_odense, 
#     left_on=['ISIN', 'Beholdning'], 
#     right_on=['ISIN kode', 'Værdipapirets navn'], 
#     how='inner'  # 'inner' ensures only matching rows are kept
# )

# df_merged


,ISIN,Beholdning,Markedsværdi (DDK),ISIN kode,Værdipapirets navn,Markedsværdi (DKK)
0,XS2794589403,5.13% British Telecommunications PLC 2054,392800,XS2794589403,5.13% British Telecommunications PLC 2054,392800.0
1,XS2791972248,3.76% JPMorgan Chase & Co 2034,1362700,XS2791972248,3.76% JPMorgan Chase & Co 2034,1362700.0
2,XS2790910272,3.50% Banco Bilbao Vizcaya Argentaria SA 2031,730600,XS2790910272,3.50% Banco Bilbao Vizcaya Argentaria SA 2031,730600.0
3,XS2790333889,3.96% Morgan Stanley 2035,871400,XS2790333889,3.96% Morgan Stanley 2035,871400.0
4,XS2790333707,3.79% Morgan Stanley 2030,1245700,XS2790333707,3.79% Morgan Stanley 2030,1245700.0
...,...,...,...,...,...,...
3528,XS2264978623,United Overseas Bank Ltd 0.01% 01-12-2027,3875012,XS2264978623,United Overseas Bank Ltd 0.01% 01-12-2027,3875012.0
3529,CH0506071387,Valiant Bank AG 0.000000% 20-01-2026,487142,CH0506071387,Valiant Bank AG 0.000000% 20-01-2026,487142.0
3530,SK4000024923,Vseobecna Uverova Banka AS 3.25% 20-03-2031,303887,SK4000024923,Vseobecna Uverova Banka AS 3.25% 20-03-2031,303887.0
3531,XS2388390507,Westpac Banking Corp 0.01% 22-09-2028,730929,XS2388390507,Westpac Banking Corp 0.01% 22-09-2028,730929.0


In [46]:
# Find duplicates based on 'Markedsværdi (DDK)' and 'Markedsværdi (DKK)' columns
duplicates = df_merged[df_merged.duplicated(subset=['ISIN', 'Beholdning'], keep=False)]

# Display the duplicate rows
print(duplicates)


              ISIN                                         Beholdning  \
261   US9426222009                                         Watsco Inc   
262   US9426222009                                         Watsco Inc   
264   US92338C1036                                       Veralto Corp   
265   US92338C1036                                       Veralto Corp   
268   US8740391003          Taiwan Semiconductor Manufacturing Co Ltd   
...            ...                                                ...   
3444  DK0009538225  Nykredit Realkredit FRN 10-01-2025 IO CIBOR 3M...   
3452  DK0009819310                                         SKBV 24/II   
3453  DK0009819310                                         SKBV 24/II   
3454  DK0009819583                                        SKBV 24/III   
3455  DK0009819583                                        SKBV 24/III   

     Markedsværdi (DDK)     ISIN kode  \
261             4410400  US9426222009   
262             4410400  US9426222009   


In [42]:
import pandas as pd

# Define a function to calculate the difference and handle errors
def calculate_difference(row):
    try:
        # Try converting both columns to float
        value1 = float(row['Markedsværdi (DDK)'])
        value2 = float(row['Markedsværdi (DKK)'])
        
        # Return the difference
        return value1 - value2
    except (ValueError, TypeError):  # Handle conversion errors
        return 'FEJL'

# Apply the function to each row to create the new 'Forskel' column
df_merged['Forskel'] = df_merged.apply(calculate_difference, axis=1)

# Display the first few rows of the dataframe to verify the result
print(df_merged[['Markedsværdi (DDK)', 'Markedsværdi (DKK)', 'Forskel']].head())

df_merged.to_excel('forskel_i_odense_data.xlsx')


  Markedsværdi (DDK)  Markedsværdi (DKK) Forskel
0             392800            392800.0     0.0
1            1362700           1362700.0     0.0
2             730600            730600.0     0.0
3             871400            871400.0     0.0
4            1245700           1245700.0     0.0


In [11]:
df_odense_ok.head()
df_odense_ok['Markedsværdi (DKK)'] = df_odense_ok['Markedsværdi (DDK)']

In [15]:
# Alternatively, you can just check the types directly in the column
invalid_rows = df_odense_ok[df_odense_ok['Markedsværdi (DKK)'].apply(lambda x: isinstance(x, datetime.time))]
print(invalid_rows)

              ISIN         Beholdning Markedsværdi (DDK) Unnamed: 3  \
2646  US1255231003    Cigna Group/The    05:16:46.320000        NaN   
2652  US1266501006         CVS Health    01:21:43.040000        NaN   
2664  US4781601046  Johnson & Johnson    01:30:21.770000        NaN   
2861  US67066G1040             NVIDIA    15:16:08.720000        NaN   

      Unnamed: 4 Unnamed: 5 Markedsværdi (DKK)  
2646         NaN     Aktier    05:16:46.320000  
2652         NaN     Aktier    01:21:43.040000  
2664         NaN     Aktier    01:30:21.770000  
2861         NaN     Aktier    15:16:08.720000  


In [18]:
# Remove rows where 'Markedsværdi (DKK)' contains datetime.time
df_odense_ok = df_odense_ok[~df_odense_ok['Markedsværdi (DKK)'].apply(lambda x: isinstance(x, datetime.time))]

# Replace '-' with NaN (Fjerner dem, hvor der ikke er værdi. Det er fx to fra Odense)
df_odense_ok['Markedsværdi (DKK)'] = df_odense_ok['Markedsværdi (DKK)'].replace('-', np.nan)

# Remove any potential commas, spaces, or other non-numeric characters
df_odense_ok['Markedsværdi (DKK)'] = df_odense_ok['Markedsværdi (DKK)'].replace({',': '', ' ': ''}, regex=True)

# Convert the column to float
df_odense_ok['Markedsværdi (DKK)'] = df_odense_ok['Markedsværdi (DKK)'].astype(float)

In [21]:
# Calculate the sum of the 'Markedsværdi (DKK)' column
total_sum = df_odense_ok['Markedsværdi (DKK)'].sum()

# Display the result
print(f"Total sum of 'Markedsværdi (DKK)': {total_sum}")


Total sum of 'Markedsværdi (DKK)': 3133739744.73


In [22]:
# Calculate the sum of the 'Markedsværdi (DKK)' column
total_sum = df_odense['Markedsværdi (DKK)'].sum()

# Display the result
print(f"Total sum of 'Markedsværdi (DKK)': {total_sum}")


Total sum of 'Markedsværdi (DKK)': 3129622300.1900005


In [23]:
df_odense_ok[df_odense_ok['ISIN'] =='DK0004619038']

,ISIN,Beholdning,Markedsværdi (DDK),Unnamed: 3,Unnamed: 4,Unnamed: 5,Markedsværdi (DKK)
482,DK0004619038,1.50% Realkredit Danmark A/S 2053,041848,NaN,NaN,NaN,41848.0


In [28]:
type(df_odense_ok[df_odense_ok['ISIN'] =='DK0004619038']['Markedsværdi (DDK)'].iloc[0])

str

In [52]:
df_odense_ok[df_odense_ok['ISIN'] =='CH0038863350']

,ISIN,Beholdning,Markedsværdi (DDK)
306,CH0038863350,Nestle SA,5760400
1007,CH0038863350,Nestle SA,564192.85
1896,CH0038863350,NESTLE SA-REG,405658.76
2677,CH0038863350,Nestle,161255.23
2850,CH0038863350,Nestle,365224.03


In [54]:
df_odense[df_odense['ISIN kode'] =='CH0038863350']

,ISIN kode,Værdipapirets navn,Markedsværdi (DKK)
7733,CH0038863350,Nestle,161255.23
7734,CH0038863350,Nestle,365224.03
7892,CH0038863350,Nestle SA,564192.85
7893,CH0038863350,Nestle SA,5760400.00
7912,CH0038863350,NESTLE SA-REG,405658.76
